In [1]:
%load_ext dotenv
%dotenv

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col

spark = SparkSession.builder.appName("Trip Data").master("local[*]").getOrCreate()
spark.sparkContext.setLogLevel("FATAL")

dbname = "tripdata"
tblname = "ny_taxi"
ny_taxi = spark.sql(f'SELECT * FROM {dbname}.{tblname}')
ny_taxi = ny_taxi.withColumn('trip_duration', col('lpep_dropoff_datetime').cast('long') - col('lpep_pickup_datetime').cast('long'))
ny_taxi.select('trip_duration', 'trip_distance', 'total_amount').summary().show()

SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/usr/share/aws/emr/emrfs/lib/slf4j-log4j12-1.7.12.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/usr/lib/spark/jars/slf4j-log4j12-1.7.30.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.slf4j.impl.Log4jLoggerFactory]


+-------+-----------------+-----------------+------------------+
|summary|    trip_duration|    trip_distance|      total_amount|
+-------+-----------------+-----------------+------------------+
|  count|            20000|            20000|             20000|
|   mean|          1602.87|3.301526999999994| 15.86084999999584|
| stddev|8265.798653153171|3.194486083134158|11.168999261559875|
|    min|                0|              0.0|             -60.0|
|    25%|              360|              1.2|              8.75|
|    50%|              660|             2.23|             12.36|
|    75%|             1020|              4.3|              19.8|
|    max|            86400|             46.0|             240.0|
+-------+-----------------+-----------------+------------------+



In [3]:
ny_taxi.filter(col('total_amount') < 0).select('total_amount').count()

50